In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.tracers.langchain import wait_for_all_tracers
from langchain.callbacks.tracers import LangChainTracer
from langsmith import Client

client = Client()

tracer = LangChainTracer(project_name="new-project-langsmith-101")
import uuid
prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a truthful AI assistant,"),
        ("user", "{input}")
    ])

llm_model = ChatOpenAI(temperature=0)

output_parser = StrOutputParser()

chain = (prompt | llm_model | output_parser).with_config(
    {
        "tags": ["openai-chat", "simple-chain"],
        "metadata": {"user_id": str(uuid.uuid4())},
        "run_name": "chat-openai-chain",
    })

In [ ]:
# Basic approach to call the chain
################################################
response = chain.invoke({"input":"What is the best way to work with Large Language Models?"})

In [5]:
# Highly recommended approach to call the chain
################################################
from langchain import callbacks
try:
    with callbacks.collect_runs() as cb:
        response = chain.invoke({"input":"What is the best way to work with Large Language Models?"},
                    config= {"callbacks": [tracer]},
                #  {"tags": ["openai-chat", "simple-chain"],
                #   "metadata": {"user_id": str(uuid.uuid4())}}
                )
        run_id = cb.traced_runs[0].id
        
        client.create_feedback(
            run_id=run_id,
            key="user-rating",
            score=0.8,
            comment="The response was helpful."
        )
finally:
    wait_for_all_tracers()


In [6]:
run_id

UUID('a3a444fe-817d-4a03-bfe8-04d791d36979')

In [8]:
response

"Working with Large Language Models can be a complex task, but here are some best practices to help you effectively work with them:\n\n1. Data Preparation: Ensure that your training data is clean, relevant, and diverse to help the model learn patterns effectively.\n\n2. Model Selection: Choose the right pre-trained language model that fits your specific use case and fine-tune it on your dataset if necessary.\n\n3. Fine-Tuning: Fine-tune the pre-trained model on your specific task or domain to improve its performance and adapt it to your needs.\n\n4. Hyperparameter Tuning: Experiment with different hyperparameters such as learning rate, batch size, and optimizer to optimize the model's performance.\n\n5. Evaluation: Evaluate the model's performance using appropriate metrics and test datasets to ensure it meets your requirements.\n\n6. Monitoring: Continuously monitor the model's performance and make adjustments as needed to maintain its accuracy and effectiveness.\n\n7. Ethical Consider